In [ ]:
from circle_fit import least_squares_circle as lsc
from src.tools import getFrame, C, np
from matplotlib import pyplot as plt
from scipy.stats import mode
from os import listdir as L

index = 12

EDGE_TOL = 400
RADIUS_RANGE = (1, 20)
SCAN_STEP = 8
REGEN_TOL = 0.1
CANNY_STRENGTH = 2.9
VIDEO_FILE = './data/Kani_2/' + L('./data/Kani_2')[index]
print(VIDEO_FILE)

RED = (255, 0, 0)

In [ ]:
img = getFrame(VIDEO_FILE, 1)

img2 = img.copy()
width, height = img.shape[:2]

# first one is up and down | second one is left and right
cx, cy = round(width * 0.432), round(height * 0.83)
# cx, cy = round(width * 0.45), round(height * 0.84)

pct = round(width * 0.035)
x0, y0, x1, y1 = cx - pct, cy - pct, cx + pct, cy + pct

C.rectangle(img2, (y0, x0), (y1, x1), RED, 2)
plt.imshow(img2)
plt.show()

In [ ]:
ref = img[x0:x1, y0:y1]

cs = CANNY_STRENGTH*100
edges = C.Canny(ref, cs, cs)

cx, cy, R, _ = lsc(np.argwhere(edges > 0))
cx, cy, R = int(cx), int(cy), int(R)

frame = ref[round(cx - R):round(cx + R), round(cy - R):round(cy + R)]
plt.imshow(frame, cmap='gray')
plt.savefig('frame.png', bbox_inches='tight')
plt.show()

In [ ]:
res = C.matchTemplate(img, frame, C.TM_CCOEFF_NORMED)
loc = np.where(res >= 0.68) #1std

cxy = []
for pt in zip(*loc[::-1]):
  x, dx = pt[0], frame.shape[1]
  y, dy = pt[1], frame.shape[0]
  y = round(y/R)*R

  cxy.append((round(x+dx/2), round(y+dy/2)))
  C.rectangle(img, pt, (x+dx, y+dy), RED, 1)
cxy = np.array(cxy)

Y = mode(cxy[:, 1]).mode
cxy = cxy[cxy[:, 1] == Y]
diffs = np.diff(cxy[:, 0], 1)
mean = np.mean(diffs)

plt.imshow(img)
plt.show()

In [ ]:
print(f"With mean diff of {mean:.2f}R @ R={R}")

In [67]:
data_dict = {
  'index': str(index),
  'R': str(R),
  'Y': str(Y),
  'D': str(mean.round(2)),
  'sz': str(frame.shape),
  'file': L('./data/Kani_2')[index],
  'bsz': str((y0, x0, y1, x1)),
}

with open('./data.txt', 'w') as f:
  from json import dumps
  f.write(str(dumps(data_dict)))

In [68]:
!rm -rf ./in_dir/* ./out_dir/*

In [ ]:
!ffmpeg -hide_banner -loglevel error -i "{VIDEO_FILE}" './in_dir/%03d.png'
!python3 src/extract.py in_dir out_dir {max_Y}
!rm -r ./out_dir/0.png ./out_dir/000.png
!ls ./out_dir | wc -l